In [1]:
from typing import TypedDict, Any, List, Annotated
import operator


from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:

class AnyMessage:
    pass

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]


def add_message(agent_state: AgentState, message: AnyMessage):
    agent_state['messages'] = agent_state.get('messages', []) + [message]
    return agent_state

# Example usage
agent_state = AgentState(messages=[])
message = AnyMessage()
new_agent_state = add_message(agent_state, message)
print(new_agent_state)

{'messages': [<__main__.AnyMessage object at 0x1230f7250>]}


In [3]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [4]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [5]:

from dashscope import Generation
import os
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(
        api_key=os.getenv("DASHSCOPE_API_KEY"), # 如果您没有配置环境变量，请在此处用您的API Key进行替换
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", # 填写DashScope base_url
        model="qwen-max"
        )
abot = Agent(model, [tool], system=prompt)

In [7]:
messages = [HumanMessage(content="what is the weather in dalian")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in dalian'}, 'id': ''}
Back to the model!


BadRequestError: Error code: 400 - {'error': {'code': 'invalid_type', 'param': 'messages[0].content', 'message': "Invalid type for 'messages[0].content': expected one of a string or array of objects, but got an object instead.", 'type': 'invalid_request_error'}}

In [14]:
result

{'messages': [HumanMessage(content='What is the weather in sf?'),
  AIMessage(content='Let me check the current weather in San Francisco for you.', additional_kwargs={'tool_calls': [{'id': '', 'function': {'arguments': '{"query": "current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 277, 'total_tokens': 313}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-66f46c77-be3e-46a8-92ba-3b37ab15c84b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': ''}], usage_metadata={'input_tokens': 277, 'output_tokens': 36, 'total_tokens': 313}),
  ToolMessage(content='[{\'url\': \'https://www.wunderground.com/hourly/us/ca/san-francisco/KCASANFR2002/date/2024-6-20\', \'content\': \'Current Weather for Popular Cities . San Francisco, CA warning 53 ° F Clear;

In [15]:
result['messages'][-1].content

"The current weather in San Francisco is sunny with a temperature of 55.5°F (or 13.0°C). The wind is blowing at 12.1 mph coming from the west-southwest direction, and the humidity is 81%. It was last updated at 19:15 local time. \n\nHere's a link for more details: [Current Weather in San Francisco](https://www.wunderground.com/hourly/us/ca/san-francisco/KCASANFR2002/date/2024-6-20)"

In [17]:
from openai import OpenAI
import os

def get_response():
    client = OpenAI(
        api_key=os.getenv("DASHSCOPE_API_KEY"), # 如果您没有配置环境变量，请在此处用您的API Key进行替换
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 填写DashScope SDK的base_url
    )
    completion = client.chat.completions.create(
        model="qwen-max",
        messages=[{'role': 'system', 'content': 'You are a helpful assistant.'},
                  {'role': 'user', 'content': '大连什么时候会下雨'}]
        ),
    tools=[tool]
    print(f"completion: {completion}")
    # print(completion.model_dump_json())
get_response()

completion: (ChatCompletion(id='chatcmpl-15b0eb74-c138-96f0-9244-5a106479ba06', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='作为一个AI模型，我无法实时提供天气信息。要查询大连或其他任何地区的最新天气预报，包括降雨情况，建议您访问中国气象局官方网站、使用手机上的天气应用程序或者搜索在线天气服务，这些平台能提供最新的天气预报信息。', role='assistant', function_call=None, tool_calls=None))], created=1718848663, model='qwen-max', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=53, prompt_tokens=23, total_tokens=76)),)


In [30]:
from langchain_openai import ChatOpenAI
import os

def get_response():
    llm = ChatOpenAI(
        api_key=os.getenv("DASHSCOPE_API_KEY"), # 如果您没有配置环境变量，请在此处用您的API Key进行替换
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", # 填写DashScope base_url
        model="qwen-max"
        )
    messages = [
        {"role":"system","content":"You are a helpful assistant."}, 
        {"role":"user","content":"你的模型名字是什么"}
    ]
    response = llm.invoke(messages)
    print(response.json(ensure_ascii=False))

if __name__ == "__main__":
    get_response()
    

{"content": "我是来自阿里云的语言模型，我叫通义千问。", "additional_kwargs": {}, "response_metadata": {"token_usage": {"completion_tokens": 14, "prompt_tokens": 23, "total_tokens": 37}, "model_name": "qwen-max", "system_fingerprint": null, "finish_reason": "stop", "logprobs": null}, "type": "ai", "name": null, "id": "run-2deb8f49-64aa-4971-9c70-53971d354cbe-0", "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": {"input_tokens": 23, "output_tokens": 14, "total_tokens": 37}}
